In [1]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [4]:
# Path ke GeckoDriver, sesuaikan dengan path di komputermu
gecko_path = "D:/geckodriver.exe"  # Sesuaikan path ke GeckoDriver
service = FirefoxService(executable_path=gecko_path)

# Path ke Firefox executable
firefox_path = "C:/Program Files/Mozilla Firefox/firefox.exe"  # Lokasi Firefox.exe


In [5]:
# Konfigurasi Firefox
options = Options()
options.binary_location = firefox_path  # Menetapkan lokasi binary Firefox
options.headless = False  # Set ke True jika Anda tidak ingin membuka jendela browser


def create_driver():
    options = Options()
    options.binary_location = 'C:/Program Files/Mozilla Firefox/firefox.exe'
    service = FirefoxService(executable_path='D:/geckodriver.exe')
    return webdriver.Firefox(service=service, options=options)


In [6]:

def get_tweet(element):
    try:
        user = element.find_element(By.XPATH, ".//*[contains(text(), '@')]").text
        text = element.find_element(By.XPATH, ".//div[@lang]").text
        date = element.find_element(By.XPATH, ".//time").get_attribute("datetime")

        try:
            reply_to_user = element.find_element(By.XPATH, ".//div[@dir='ltr']//a[contains(@href, '/')]").text
        except:
            reply_to_user = None

        try:
            tweet_link = element.find_element(By.XPATH, ".//a[@href and contains(@href, '/status/')]").get_attribute("href")
        except:
            tweet_link = None

        try:
            media_links = [media.get_attribute("src") for media in element.find_elements(By.XPATH, ".//img[contains(@src, 'media')]")]
        except:
            media_links = []

        tweet_data = [user, text, date, tweet_link, media_links, reply_to_user]
        return tweet_data
    except Exception as e:
        print(f"Error: {e}")
        return None

def click_retry_button(driver):
    try:
        retry_button = driver.find_element(By.XPATH, "//div[text()='Retry']/parent::div")
        retry_button.click()
        print("Clicked the retry button")
        WebDriverWait(driver, 15).until(EC.invisibility_of_element_located((By.XPATH, "//div[text()='Retry']")))
    except:
        pass

def scrape_tweets(driver):
    user_data = []
    text_data = []
    date_data = []
    tweet_link_data = []
    media_link_data = []
    reply_to_data = []
    tweet_ids = set()

    scrolling = True
    while scrolling and len(user_data) < 10000:
        try:
            print("Searching for tweets...")
            tweets = WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.XPATH, "//article")))
            print(f"Number of tweets found: {len(tweets)}")
            for tweet in tweets:
                tweet_list = get_tweet(tweet)
                if tweet_list:
                    tweet_id = ''.join(tweet_list[:2])
                    if tweet_id not in tweet_ids:
                        tweet_ids.add(tweet_id)
                        user_data.append(tweet_list[0])
                        text_data.append(" ".join(tweet_list[1].split()))
                        date_data.append(tweet_list[2])
                        tweet_link_data.append(tweet_list[3])
                        media_link_data.append(tweet_list[4])
                        reply_to_data.append(tweet_list[5])

            last_height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            WebDriverWait(driver, 30).until(lambda driver: driver.execute_script("return document.body.scrollHeight") > last_height)
            new_height = driver.execute_script("return document.body.scrollHeight")

            click_retry_button(driver)

            if new_height == last_height:
                scrolling = False
            else:
                last_height = new_height
                time.sleep(5)  # Tambahkan penundaan yang lebih lama

        except Exception as e:
            print(f"Error during scrolling or tweet retrieval: {e}")
            break

    return user_data, text_data, date_data, tweet_link_data, media_link_data, reply_to_data

driver = create_driver()
web = 'https://x.com/search?q=point%20coffee%20lang%3Aid%20until%3A2024-07-01%20since%3A2023-07-01&src=typed_query&f=live' # Masukan link yang akan di cari
driver.get(web)
time.sleep(15)
driver.save_screenshot('screenshot_before_search.png')

user_data, text_data, date_data, tweet_link_data, media_link_data, reply_to_data = scrape_tweets(driver)

driver.save_screenshot('screenshot_after_search.png')
driver.quit()

df = pd.DataFrame({
    'User': user_data,
    'Text': text_data,
    'Date': date_data,
    'Tweet Link': tweet_link_data,
    'Media Links': media_link_data,
    'Reply To': reply_to_data
})

print(df)

Searching for tweets...
Number of tweets found: 6
Error: Message: Unable to locate element: .//*[contains(text(), '@')]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Searching for tweets...
Number of tweets found: 23
Error: Message: Unable to locate element: .//*[contains(text(), '@')]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElem

InvalidSessionIdException: Message: WebDriver session does not exist, or is not active
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
InvalidSessionIDError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:448:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.session@chrome://remote/content/shared/webdriver/Assert.sys.mjs:37:4
despatch@chrome://remote/content/marionette/server.sys.mjs:315:19
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [ ]:
import re

def extract_mentions(text):
    mentions = re.findall(r'@(\w+)', text)
    return mentions

df['Target'] = df['User']
df['Mentions'] = df['Text'].apply(extract_mentions)

df

,User,Text,Date,Tweet Link,Media Links,Reply To,Target,Mentions
0,@shakintud,eh sedih deh gatau kenapa rasa kopi kenangan s...,2024-06-30T17:27:41.000Z,https://x.com/shakintud/status/180746604756356...,[],None,@shakintud,[]
1,@miwweng,butuh kopken tolong butuh kopken hidup ini tan...,2024-06-30T08:12:03.000Z,https://x.com/miwweng/status/1807326217663922450,[],None,@miwweng,[]
2,@luoyiu,Oatside kopi kenangan mantan punya kopken enak...,2024-06-28T19:27:04.000Z,https://x.com/luoyiu/status/1806771315355140465,[],@convomf,@luoyiu,[]
3,@kalettapple,saran menu kopken selain kopi kenangan mantan pls,2024-06-28T06:57:28.000Z,https://x.com/kalettapple/status/1806582674401...,[],None,@kalettapple,[]
4,@imnotxxning,"kopken, tim kopi kenangan mantan tapi gula are...",2024-06-28T02:08:08.000Z,https://x.com/imnotxxning/status/1806509859250...,[],None,@imnotxxning,[]
...,...,...,...,...,...,...,...,...
477,@Afrisbayura,Promo Line bank di kopken (kopi kenangan) udah...,2023-07-12T12:17:03.000Z,https://x.com/Afrisbayura/status/1679102569233...,[https://pbs.twimg.com/media/F01e4nYakAAJIiC?f...,None,@Afrisbayura,[]
478,@ourrland,Beliin kopi dong kopken kenangan mantan 1 yg r...,2023-07-09T11:55:38.000Z,https://x.com/ourrland/status/1678010015754629126,[],None,@ourrland,[]
479,@darlingy0uuu,stress bangwt orang rumah gue kerjaannya minum...,2023-07-07T06:15:06.000Z,https://x.com/darlingy0uuu/status/167719954193...,[],None,@darlingy0uuu,[]
480,@nsptianaaa,Kopi kenangan gamau kasih aku voucher apa yaa....,2023-07-06T14:09:06.000Z,https://x.com/nsptianaaa/status/16769564406685...,[],None,@nsptianaaa,[Kopikenangan_ID]


In [ ]:
# Simpan kedalam csv
df.to_csv('hasil-crawling2.csv', index=False, sep=";")